In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Definer variabler der bruges til at filtrere data med.

In [3]:
score = pd.read_excel('UdstyrsSkoreMatrix.xlsx', sheet_name='Matrix', index_col =0)
score.head()
score.loc['Accelerometer','TekniskLevetid']

50

In [4]:
NB = pd.to_datetime('01-01-2023') #indflytningsdato for NordBlok NB
#NB =pd.Timestamp('21-11-2023')

MA = 12 #Maksimal alder i år ved indflytningen
 
NBMA = NB.replace(year=NB.year - MA)#dato for hvornår udstyr der kan flyttes i NordBlokken

BG = 5000 #BagatelGrænse for pris

EJER = 'Bispebjerg og Frederiksberg Hospital'

FilterInput = pd.DataFrame({'Filter funktion':['Ejer', 'Indflytnings dato', 'Bagatelgrænse i kr', 'Ekskluderede grupper'],
                           'Værdi': [EJER, NB, BG, 'nan']})

# Læs data

In [6]:
filename = 'BBH+FRH 19-03-20.xlsx' #'BBH+FRH 01-07-19.xlsx'
#filename = 'myxls.xlsx'

df = pd.read_excel(filename, sheet_name='Data') # Medusa udtræk

gz = pd.read_excel(filename, sheet_name='Gråzone') # Gråzone udstyr

xmd = pd.read_excel(filename, sheet_name='Xmodel') # Ekskluderete modeller

In [7]:
#Tilføj 
df =df.merge(score, on='Gruppenavn') #tilføj kolonne med teknisklevetid og udvidet levetid.

,App.nr,Gruppenavn,Fabrikat,Model,Serie nr.,Financiering,Indkøbspris,Genansk.pris,Indkøbt,Ejer (overafdeling),Afdeling (placering),Afsnit (Placering),Inaktiv. dato,Kassationsdato,TekniskLevetid,Forøgelse,KliniskLevetid,Fremtidssikert,Afskrivning
9399,1063824,"Analysator, immunoassay, fotometri, netdrevet",Alere,Afinion 2 Analyzer,AF20000657,Lejet,12000.0,0.0,2018-02-01,"Reumatologisk afdeling H, FRH; Bispebjerg og F...","Reumatologisk afdeling H, FRH","Reumatologisk Ambulatorieafsnit, FRH; Reumatol...",NaT,NaN,8,4,10,True,10
9400,1074210,"Blodtryksmåler, aneroidt, manuelt",Heine,GAMMA G5,1103533882,Købt,1.0,0.0,2018-10-01,Klinisk Fysiologisk Afdeling ; Bispebjerg og F...,Klinisk Fysiologisk Afdeling,"Klinisk Fys./Nuklearmedicinsk Afsnit, FRH; Kli...",NaT,NaN,8,4,10,True,10
9401,1077878,"Detektor, røntgen, digital, bærbar, batteri",Trixell,Pixium 4600,1834 DD,Købt,295000.0,295000.0,2018-12-10,Røntgenafdeling; Bispebjerg og Frederiksberg H...,Røntgenafdeling,"Røntgenafsnit, FRH; Røntgenafdeling, FRH",NaT,NaN,8,4,10,True,10
9402,1081067,"Software, patientovervågning",Philips,IntelliVue Information Center iX,2UA2490186,Betalt via forbrug,1.0,0.0,2012-03-27,Kardiologisk afdeling Y; Bispebjerg og Frederi...,"Y, Kardiologisk afdeling Y","Y21, Hjertemedicinsk sengeafsnit Y21; Y, Hjert...",NaT,NaN,8,4,10,True,10
9403,1081068,"Software, patientovervågning",Philips,IntelliVue Information Center iX,2UA249019Y,Betalt via forbrug,1.0,0.0,2012-03-27,Kardiologisk afdeling Y; Bispebjerg og Frederi...,"Y, Kardiologisk afdeling Y","Y21, Hjertemedicinsk sengeafsnit Y21; Y, Hjert...",NaT,NaN,8,4,10,True,10


In [42]:
#Tilpasninger
df['Indkøbt']= pd.to_datetime(df.Indkøbt)#konverter til datetime som Pandas kan regne på
#udregn levetider 
df['Levetid'] = df.loc[:,['Indkøbt', 'TekniskLevetid']].apply(lambda x: (x[0].replace(year = x[0].year + x[1])),axis=1)
df['UdvidetLevetid'] = df.loc[:,['Indkøbt', 'TekniskLevetid','Forøgelse']].apply(lambda x: (x[0].replace(year = x[0].year + x[1]+x[2])),axis=1)

#Find maks prisen
df["Pris"] = df[["Indkøbspris","Genansk.pris"]].max(axis=1)# find den største værdi og sæt det ind i ny kollonde "Pris"

,App.nr,Gruppenavn,Fabrikat,Model,Serie nr.,Financiering,Indkøbspris,Genansk.pris,Indkøbt,Ejer (overafdeling),...,TekniskLevetid,Forøgelse,KliniskLevetid,Fremtidssikert,Afskrivning,Pris,Årgang,LT,Levetid,UdvidetLevetid
0,14114,"Blodtomhedssystem (tourniquet), pneumatisk",Hokanson,E20 Rapid Cuff Inflator,14114,Købt,10000.0,0.0,2000-04-14,Kardiologisk afdeling Y; Bispebjerg og Frederi...,...,8,4,10,True,10,10000.0,2000.0,2008.0,2008-04-14,2012-04-14
1,1064525,"Blodtomhedssystem (tourniquet), pneumatisk",S+K,Tournipress,NaN,Købt,20000.0,0.0,1995-01-01,Dermato-Venerologisk Afd. + Sårhelingcenter; B...,...,8,4,10,True,10,20000.0,1995.0,2003.0,2003-01-01,2007-01-01
2,1059293,"Blodtomhedssystem (tourniquet), pneumatisk",Zimmer,A.T.S. 750,7510LAAW,Købt,28000.0,28000.0,2011-12-30,Ortopædkirurgisk afd. M; Bispebjerg og Frederi...,...,8,4,10,True,10,28000.0,2011.0,2019.0,2019-12-30,2023-12-30
3,1059289,"Blodtomhedssystem (tourniquet), pneumatisk",Zimmer,A.T.S. 750,7509GAAC,Købt,28000.0,28000.0,2011-12-30,Ortopædkirurgisk afd. M; Bispebjerg og Frederi...,...,8,4,10,True,10,28000.0,2011.0,2019.0,2019-12-30,2023-12-30
4,1058755,"Blodtomhedssystem (tourniquet), pneumatisk",Zimmer,A.T.S. 1200,7511JAAN,Købt,30000.0,0.0,2012-01-01,Ortopædkirurgisk afd. M; Bispebjerg og Frederi...,...,8,4,10,True,10,30000.0,2012.0,2020.0,2020-01-01,2024-01-01


# Så begynder filtereringen

In [21]:
x = df['Ejer (overafdeling)'].str.split(";", n = 1, expand = True)
df["Overafdeling"] =x[0]
df["Ejer"] = x[1]

In [22]:
fdf = df.loc[df.Ejer.str.contains(EJER)== True] #filterer på Ejer
#fdf.head()

In [23]:
#fdf = fdf.loc[~fdf.Model.isin(xmd.Model),:] # fjern ekskluderede modeller
#ffdf = fdf.loc[fdf.Model.isin(xmd.Model),:] # find ekskluderede modeller
#print(len(ffdf))

In [8]:
fdf = fdf.loc[fdf.Indkøbt - NBMA,:] #filtrer på alder
#fdf.head()

NameError: name 'fdf' is not defined

In [ ]:
fdf =fdf.loc[fdf.Pris > BG,:] #filtrer på pris
#fdf.head()

In [ ]:
fdf = fdf.loc[~fdf.Gruppenavn.isin(gz.Gruppenavn)] #fjern gråzone udstyr

In [ ]:
fdf = fdf.drop(['Indkøbspris', 'Genansk.pris', 'Ejer (overafdeling)'], axis=1)

In [ ]:
#cols = fdf.columns.tolist()
#print(cols)

In [ ]:
cols = ['App.nr', 'Gruppenavn', 'Fabrikat', 'Model', 'Serie nr.', 'Indkøbt', 'Pris', 'Overafdeling', 'Afdeling (placering)', 'Afsnit (Placering)', 'Ejer']
fdf = fdf[cols]

In [ ]:
fdf = fdf.sort_values(by=['Gruppenavn','Indkøbt'])#sorter på gruppenavn og indkøbsdato
#fdf.info()

# Opdeling i afdelinger

In [ ]:
Afd = pd.unique(fdf['Overafdeling'])
Afd.sort()

#print(Afd)

fdf['Overafdeling'] = fdf['Overafdeling'].str.slice(0,27)
afd = pd.unique(fdf['Overafdeling'])
afd.sort()

i=0
antal =[]
pris =[]
for x in afd:
    xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
    antal.append(len(xls)) # find antal udstyr i afdelingen
    pris.append(xls['Pris'].sum()) # find den samlede værdi af udstyret 
    i = i+len(xls)
   # print(x[0:28], "  ",len(xls))    
#print(i) #opsummetet antal
#antal
StatusArk =pd.DataFrame(list(zip(Afd, antal, pris)), columns =['Afdeling', 'Antal udstyr', 'Værdi'])
#StatusArk

In [ ]:
with pd.ExcelWriter('Udskiftningsbehov_Opdelt.xlsx') as writer:
    FilterInput.to_excel(writer, sheet_name='Opsumering',index = False)
    gz.to_excel(writer, sheet_name='Opsumering',startrow = 6,startcol = 1,index = False)
    StatusArk.to_excel(writer, sheet_name='Opsumering',startcol = 4, index = False)
    
    for x in afd:
        xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
        if len(xls)>0:
            xls.to_excel(writer, sheet_name=x[0:27],index = False)

# Skriv det hele i et regneark

In [ ]:
with pd.ExcelWriter('Udskiftningsbehov.xlsx') as writer:
    
    fdf.to_excel(writer,index = False)